In [16]:
import pandas as pd
from binance import Client
from prophet import Prophet

# Binance API credentials 
api_key = "5bb2b139e2f3693d8b35ca1f8252723355505d562b8e20625b513d0415f2a1e9"
api_secret = "7bbec5768461b201caba13b9605b0bc7fa0332a06f9480d5ef3405edd582ae80"

client = Client(api_key, api_secret)

# Get historical candlestick data
candlesticks = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY,  "1 Jan, 2021", "1 Jan, 2023")
# Create DataFrame from the candlestick data
df = pd.DataFrame(candlesticks, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_Time',
                                         'Quote Asset Volume', 'Number_of_Trades', 'Taker Buy Base Volume',
                                         'Taker Buy Quote Volume', 'Ignore'])

# Drop unnecessary columns
columns_to_drop = ['Quote Asset Volume', 'Taker Buy Base Volume', 'Taker Buy Quote Volume', 'Ignore']
df = df.drop(columns=columns_to_drop)

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], unit='ms')

# Convert relevant columns to numeric
cols_to_numeric = ['Open', 'High', 'Low', 'Close', 'Volume']
df[cols_to_numeric] = df[cols_to_numeric].apply(pd.to_numeric)

# Function to calculate Exponential Moving Average (EMA)
def calculate_ema(data, window):
    ema = data.ewm(span=window, adjust=False).mean()
    return ema

# Function to calculate Relative Strength Index (RSI)
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to calculate Bollinger Bands (BB)
def calculate_bb(data, window=14):
    rolling_mean = data.rolling(window=window).mean()
    rolling_std = data.rolling(window=window).std()
    upper_band = rolling_mean + (2 * rolling_std)
    lower_band = rolling_mean - (2 * rolling_std)
    return upper_band, lower_band

# Calculate EMA, RSI, and Bollinger Bands
def calculate_indicators(df):
    df['EMA'] = calculate_ema(df['Close'], 55)
    df['RSI'] = calculate_rsi(df['Close'])
    df['Upper_Band'], df['Lower_Band'] = calculate_bb(df['Close'])

calculate_indicators(df)

# Implement the long and short strategies
def long_strategy(row):
    if row['RSI'] < 40 and row['Close'] > row['EMA'] and row['Close'] < row['Lower_Band']:
        return 1
    return 0

def short_strategy(row):
    if row['RSI'] > 60 and row['Close'] < row['EMA'] and row['Close'] > row['Upper_Band']:
        return -1
    return 0

df['Buy_Signal'] = df.apply(long_strategy, axis=1)
df['Sell_Signal'] = df.apply(short_strategy, axis=1)

# Prepare data for Prophet
prophet_df = df[['Date', 'Close']]
prophet_df = prophet_df.rename(columns={'Date': 'ds', 'Close': 'y'})

# Train the Prophet model
model = Prophet(daily_seasonality=True)
model.fit(prophet_df)

# Create future dataframe for making predictions
future = model.make_future_dataframe(periods=1)  # Predict for one more day
forecast = model.predict(future)

# Extract the predicted 'yhat' value for the next day
predicted_tomorrow_price = forecast.iloc[-1]['yhat']

# Compare with today's closing price to predict 'target'
today_closing_price = df.iloc[-1]['Close']
predicted_target = 1 if predicted_tomorrow_price > today_closing_price else 0

# Print the predicted target value
print("The predicted value for tomorrow is:", predicted_target)


12:25:29 - cmdstanpy - INFO - Chain [1] start processing
12:25:30 - cmdstanpy - INFO - Chain [1] done processing


The predicted value for tomorrow is: 0
